In [50]:
import pathlib

import lifelines
import numpy as np
import pandas as pd
import scipy
from scipy.stats import chi2

In [51]:
DATA_DIR = pathlib.Path("~/Data/CPH200B")

In [52]:
project_1_data_dir = DATA_DIR / "Project 1"
project_2_data_dir = DATA_DIR / "Project 2"

## 2.1 Warm-up Exercise: Hypothesis Testing & Confounding [6 pts]

The most basic form of causal inference involves comparing survival curves in different groups stratified by an
intervention of interest. In this task, we will implement hypothesis testing methods to examine whether differences between the outcomes of treated and untreated patients are statistically significant, and whether these
difference reflect the causal effect of the intervention.

For all the tasks below, we will use the UNOS heart transplant [1] dataset from Project 1.

In [53]:
unos_data_filepath = project_1_data_dir / "UNOS_train.csv"

unos_data = pd.read_csv(unos_data_filepath)
unos_data["event"] = unos_data['Censor (Censor = 1)'] == 0


### Task 2.1.1 [1 pts]. 
Implement the Log-Rank test from scratch in Python. Using the UNOS dataset, apply
your implemented test to check whether the survival outcomes of patients on ventricular assist device (VAD)
support differ from those of patients without VAD support.

In [56]:
def log_rank_test(time_A, time_B, event_A, event_B):
    # Combine data into a single DataFrame
    df = pd.DataFrame(
        {
            "time": np.concatenate((time_A, time_B)),
            "event": np.concatenate((event_A, event_B)),
            "group": np.concatenate(
                (np.zeros_like(time_A), np.ones_like(time_B))
            ),
        }
    )

    # Sort by time
    df = df.sort_values(by="time")

    # Compute risk set counts efficiently using cumulative sums
    df["R_A"] = (df["group"] == 0)[::-1].cumsum()[::-1]  # Risk set for group A
    df["R_B"] = (df["group"] == 1)[::-1].cumsum()[::-1]  # Risk set for group B
    df["R"] = df["R_A"] + df["R_B"]  # Total risk set

    # Events per time point
    df["d_A"] = ((df["group"] == 0) & (df["event"] == 1)).astype(int)
    df["d_B"] = ((df["group"] == 1) & (df["event"] == 1)).astype(int)
    df["d"] = df["d_A"] + df["d_B"]

    # Compute expected events
    df["E_A"] = df["d"] * (df["R_A"] / df["R"])
    df["E_B"] = df["d"] * (df["R_B"] / df["R"])

    # Compute variance contributions
    df["V"] = (df["R_A"] * df["R_B"] * df["d"] * (df["R"] - df["d"])) / (
        df["R"] ** 2 * (df["R"] - 1)
    )
    df["V"] = df["V"].fillna(0)  # Handle division by zero cases

    # Compute log-rank test statistic
    O_A, E_A, V = df["d_A"].sum(), df["E_A"].sum(), df["V"].sum()
    Z = (O_A - E_A) ** 2 / V if V > 0 else 0

    # Compute p-value from chi-square distribution (df=1)
    p_value = 1 - chi2.cdf(Z, df=1)

    return Z, p_value

In [57]:
event_col = "event"
time_col = "Survival Time"

groups = unos_data.groupby("vad_while_listed")

duration_a = groups.get_group(0)[time_col].values
duration_b = groups.get_group(1)[time_col].values

event_a = groups.get_group(0)[event_col].values
event_b = groups.get_group(1)[event_col].values

z, pval = log_rank_test(duration_a, duration_b, event_a, event_b)
print(f"Z-statistic: {z}")
print(f"P-value: {pval}")

#
pval_from_lifelines = lifelines.statistics.logrank_test(duration_a, duration_b, event_a, event_b).p_value

if np.isclose(pval, pval_from_lifelines):
    print("P-values are close.")
else:
    print("P-values are not close.")
    print(f"P-value from lifelines: {pval_from_lifelines}")

Z-statistic: 57.830055054806515
P-value: 2.853273173286652e-14
P-values are close.


### Task 2.1.2 [1 pts]. 
Propose a method to determine if there are confounders in the UNOS dataset for the effect of VAD support on survival outcomes. List all detected confounders.

In [ ]:
# find confounders


### Task 2.1.3 [2 pts]. 
For the comparison of survival curves to have a causal interpretation, we need to adjust for confounding variables that may cause the patient groups being compared to have different clinical features.

### Task 2.1.4 [2 pts]. 
Propose a propensity-weighted version of the Kaplan-Meier estimator you implemented in Project 1 that adjusts for confounding. Plot the propensity-weighted Kaplan-Meier curves in patients with and without VAD.
Compare this plot with the survival curves of both groups using the standard Kaplan-Meier estimators.

Propose a propensity-weighted version of the Long-Rank test. Apply this test to check whether the survival outcomes of patients on VAD support differ from those of patients without VAD. Com-
pare the result of this test with the unadjusted test you implemented in Task 2.1.1. Comment on the results.

## 2.2 ML-based Estimation of Average Treatment Effects [6 pts]

### 2.2.1 Clinical Background, Dataset, and Setup

In this task, we will use individual patient data from the International Stroke Trial (IST), one of the largest
randomized trials ever conducted in acute stroke [2]. The trial investigated the impact of aspirin and subcuta-
neous heparin on patients with acute ischemic stroke, with treatment randomization within 48 hours of symp-
tom onset. The trial findings indicated no effect of both aspirin and heparin on 14-day and 6-month mortality.
The trial protocol and data dictionary have been provided to you.
The original IST data lacks confounding as it was generated through a randomized trial. The instructor intro-
duced confounding artificially by filtering patients out of the trial using a random function that depends on the
patient features. The resulting dataset mimics an observational dataset where treatment is assigned through
a mechanism that depends on patient features. You will conduct the following tasks using the artificially
confounded dataset with the goal of recovering the same treatment effects estimated in the randomized trial.

Estimate the average effect of aspirin and heparin on 14-day mortality using the following estimators. Compare your estimates with those of the original trial and provide commentary on the results.

### Task 2.2.1 [1 pts]. 
A standard difference-in-means estimator.


### Task 2.2.2 [1 pts]. 
An inverse propensity weighting (IPW) estimator using a Gradient Boosting model for the propensity scores.

### Task 2.2.3 [2 pts]. 
A covariate adjustment estimator using a Gradient Boosting model with T-learner, S-learner, and X-learner architectures.

### Task 2.2.4 [2 pts]. 
An augmented IPW (doubly-robust) estimator that combines the propensity model from
Task 2.2.2 and an outcomes model based on the S-learner in Task 2.2.3

## 2.3 Counterfactual Inference and Domain Adaptation [8 pts]

In this task, we will explore the application of concepts from the machine learning literature to estimate het-
erogeneous treatment effects. The seminal work in [3] establishes a link between estimating treatment effects
and the domain adaptation problem in machine learning. Using this insight, the authors repurpose ideas from
domain adaptation literature to create a new deep learning model for estimating the conditional average treat-
ment effects (CATE) function. The core idea of their algorithm is to eliminate confounding bias by learning a
representation Φ of the features X that aligns the distribution of treated and control populations, Φ(X|T = 1)
and Φ(X|T = 0), in the representation space, referred to by the authors as a “balancing” representation

Please read the paper carefully and complete the following tasks.

### Task 2.3.1 [3 pts]. 

Implement the T ARN et and CF RM M D models proposed in [3] in PyTorch. Evaluate
the performance of all models using the semi-synthetic benchmark dataset included in the Project 2 notebook.

### Task 2.3.2 [1 pts]. 

Visualize the treated and control features before and after applying the balancing representation Φ(.) using t-SNE. Comment on the results.

### Task 2.3.3 [1 pts]. 

Show the impact of the scaling parameter α (Eq. (3) in [3]) on the loss function on the
test set for the Maximum Mean Discrepancy (MMD) regularizer.

### Task 2.3.4 [3 pts].

Use the TARNet and CFR<sub>MMD</sub> models to estimate average treatment effects using
the IST data in Task 2.2. Assess the alignment of your estimates with the trial results and compare them
to the estimators in Tasks 2.2.3 and 2.2.4.

## 2.4 NeurIPS Reviewer for a Day: Reviewing & Reproducing Recent Research on ML-Based Causal Inference [10 pts]
In this task, we will focus on one paper that proposes new methods for estimating CATE inspired by ideas we studied in Lectures 7, 8 and 9. The paper is “Adapting Neural Networks for the Estimation of Treatment
Effects” by Claudia Shi, David Blei and Victor Veitch, which was published in NeurIPS 2019. The objective of this task is to develop critical paper review skills and practice reproducing research results. Please read
the paper carefully and complete the following tasks.

### Task 2.4.1 [5 pts]. 

Please review the NeurIPS 2024 reviewing guidelines and write a comprehensive review of this paper in accordance with those guidelines.



### Task 2.4.2 [5 pts]. 

Implement the DragonNet and Targeted regularization methods proposed in this paper in PyTorch and reproduce their performance results on the IHDP dataset (Table 1 in the paper)